In [ ]:
/from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
from skimage.measure import label, regionprops
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# Function to convert binary mask to list of ROIs
def mask_to_rois(mask):
    labeled_array = label(mask)
    regions = regionprops(labeled_array)
    rois = [{'min_row': region.bbox[0], 'min_col': region.bbox[1], 'max_row': region.bbox[2], 'max_col': region.bbox[3]}
            for region in regions]
    return rois

# Function to process a single file and return the number of ROIs
def process_file(file_path):
    # Load cytoplasmic mask from .tif file using OpenCV
    cyto_mask = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)

    # Convert mask to ROIs
    rois = mask_to_rois(cyto_mask)

    # Count number of ROIs in the current image
    num_rois = len(rois)
    return num_rois

# Function to process a single folder and return the total number of ROIs
def process_folder(folder_path):
    # List all .tif files in the folder
    cyto_mask_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".tif")]

    total_rois = 0

    # Process files in parallel using ThreadPoolExecutor
    max_workers = min(32, os.cpu_count() + 4)  # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(process_file, cyto_mask_files))

    total_rois = sum(results)
    return total_rois

# List of folder paths to analyze
folder_paths = [
    "/content/drive/MyDrive/Cellpose/data/SK/day_1_sliced/Masks/",
    "/content/drive/MyDrive/Cellpose/data/SK/day_4_sliced/Masks/",
    "/content/drive/MyDrive/Cellpose/data/SK/day_7_sliced/Masks",
    # Add more folder paths as needed
]

# Process each folder and store results
results = []
for folder_path in folder_paths:
    num_rois = process_folder(folder_path)
    results.append({'folder': folder_path, 'num_rois': num_rois})

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Export the results to a CSV file
output_csv = "/content/drive/MyDrive/Cellpose/data/roi_summary.csv"
df.to_csv(output_csv, index=False)

print(f"Summary of ROIs has been saved to {output_csv}")


Summary of ROIs has been saved to /content/drive/MyDrive/Cellpose/data/roi_summary.csv
